In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('/Users/jayasri/Desktop/crop data.csv')

In [27]:
df

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee


In [28]:
df.isnull().sum()

N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64

In [29]:
df.columns

Index(['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label'], dtype='object')

In [30]:
df = df[~(df['label']=='rice')]

In [31]:
df['label'].unique()

array(['maize', 'chickpea', 'kidneybeans', 'pigeonpeas', 'mothbeans',
       'mungbean', 'blackgram', 'lentil', 'pomegranate', 'banana',
       'mango', 'grapes', 'watermelon', 'muskmelon', 'apple', 'orange',
       'papaya', 'coconut', 'cotton', 'jute', 'coffee'], dtype=object)

In [32]:
from sklearn.model_selection import train_test_split

In [33]:


# Import necessary libraries
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import joblib 

# Assume you have features X and target y
X = df.drop('label', axis=1)  # Replace 'target_column' with your target column name
y = df['label']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize XGBoost classifier for multiclass classification
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Multiclass classification
    num_class=len(label_encoder.classes_),  # Number of unique classes
    eval_metric='mlogloss',  # Logarithmic loss for multiclass classification
    use_label_encoder=False,
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6
)

# Fit the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



/Users/jayasri/Documents/GitHub/Crop_rec_system/env/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [15:26:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9976

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00        18
           3       1.00      1.00      1.00        17
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        22
           6       1.00      1.00      1.00        14
           7       1.00      1.00      1.00        18
           8       0.95      1.00      0.97        18
           9       1.00      1.00      1.00        26
          10       1.00      1.00      1.00        17
          11       1.00      1.00      1.00        21
          12       1.00      1.00      1.00        20
          13       1.00      1.00      1.00        24
          14       1.00      1.00      1.00        26
          15       1.00      1.00      1.00        27
          16       1.00      1.00      1

In [34]:

# Save the trained model to a file
model_filename = 'xgboost_model.pkl'
joblib.dump(xgb_model, model_filename)

# Save the label encoder to a file
label_encoder_filename = 'label_encoder.pkl'
joblib.dump(label_encoder, label_encoder_filename)

# Prediction on test set
y_pred = xgb_model.predict(X_test)
y_pred_labels = label_encoder.inverse_transform(y_pred)


### PREDICTION PIPELINE ###

# Load the saved model and label encoder
loaded_model = joblib.load(model_filename)
loaded_label_encoder = joblib.load(label_encoder_filename)

# New data to predict (replace with actual new data)
# Ensure that the new_data has the same feature columns as the training data
new_data = pd.DataFrame({
    'N': [100],  # Example values; replace with actual test values
    'P': [71],
    'K': [54],
    'temperature': [22.6136],
    'humidity': [63.6907],
    'ph': [5.7499],
    'rainfall': [87.7595]
})

# Use the loaded model to predict the label for new data
new_pred = loaded_model.predict(new_data)

# Decode the prediction from numerical labels to original labels
decoded_pred = loaded_label_encoder.inverse_transform(new_pred)
print(f"Predicted Class: {decoded_pred[0]}")

Predicted Class: banana
